In [46]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
from keras.utils import np_utils
import pandas as pd
import random

# Data preprocessing

In [47]:
fr = open('train.txt') # this txt contains the filename of images with its label(numbers). here are 100 categories
lines = fr.readlines()
fr.close()
des_list = []
i = 0 # 2725 train images
for line in lines:
    i += 1
    print(i) 
    name,label = line.split()
    img = cv2.imread('train/' + name,0)
    img = cv2.GaussianBlur(img,(5,5),3)
    sift = cv2.xfeatures2d.SIFT_create() 
    kp = sift.detect(img,None)
    kp,des = sift.compute(img,kp)
    des_list.append((name,des,label)) # get image name, sift saclars and label

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


In [51]:
len(des_list),des_list[0],des_list[3][1].shape # each image has different nums of sift vectors, the length of one sift vector is 128

(2725,
 ('2934349b033b5bb5f4a305393fd3d539b700bc84.jpg',
  array([[ 44.,   0.,   0., ...,   0.,   0.,   0.],
         [  6.,   1.,  17., ...,   0.,   0.,   0.],
         [ 59.,   5.,   0., ...,   0.,   0.,   0.],
         ..., 
         [ 19.,   0.,   0., ...,  13.,   1.,   2.],
         [  8.,   0.,   0., ...,   0.,   0.,   0.],
         [ 42.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32),
  '1'),
 (121, 128))

In [52]:
# gathering all sift vectors in all images
descriptors = des_list[0][1]
i = 0
for name,descriptor,label in des_list[1:]:
    i += 1
    print(i)
    descriptors = np.vstack((descriptors,descriptor))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


# cluster sift vectors

In [53]:
num_words = 1000
km_cluster = KMeans(n_clusters = num_words)
km_cluster.fit(descriptors)

F:\anaconda\envs\tensorflow\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=1000, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [54]:
# get feature vector and one-hot label vector of each image
# the feature vector is the distribution of sift vectors in 1000 categories created by kmeans of each image 
feat_vec = []
label_vec = []
c = 0
for a in des_list:
    c += 1
    print(c)
    cat_counts = {}
    cat = km_cluster.predict(a[1])
    for i in range(cat.shape[0]):
        cat_counts[cat[i]] = cat_counts.get(cat[i],0) + 1
    vec = [0] * num_words
    for b in cat_counts.keys():
        vec[b] = cat_counts[b]
    feat_vec.append(vec)
    label_vec.append(int(a[2]) - 1)
label_vec = np_utils.to_categorical(label_vec)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087


In [55]:
# show feature vector of each image
feat_vec = np.array(feat_vec)
label_vec = np.array(label_vec)
feat_vec.shape,label_vec.shape,feat_vec[0],feat_vec[1],label_vec[0],label_vec[1]

((2725, 1000),
 (2725, 100),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# test data preprocessing

In [60]:
fr_test = open('test.txt')
lines_test = fr_test.readlines()
fr_test.close()
des_list_test = []
#lines = lines[:100]
i = 0
for line in lines_test:
    i += 1
    print(i)
    name = line[:-1]
    img = cv2.imread('test/' + name,0)
    img = cv2.GaussianBlur(img,(5,5),3)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(img,None)
    kp,des = sift.compute(img,kp)
    des_list_test.append((name,des))
    
feat_vec_test = []
for a in des_list_test:
    cat_counts = {}
    cat = km_cluster.predict(a[1])
    for i in range(cat.shape[0]):
        cat_counts[cat[i]] = cat_counts.get(cat[i],0) + 1
    vec = [0] * num_words
    for b in cat_counts.keys():
        vec[b] = cat_counts[b]
    feat_vec_test.append(vec)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# training using one layer nn

In [69]:
# here using svm normally, but nn is faster
x = tf.placeholder(dtype = tf.float32,shape = [None,num_words],name = 'input')
y = tf.placeholder(dtype = tf.float32,shape = [None,100],name = 'output')

w = tf.get_variable('weights',shape = [num_words,100],dtype = tf.float32,initializer = tf.truncated_normal_initializer(stddev = 0.1))
b = tf.get_variable('bias',shape = [100],dtype = tf.float32,initializer = tf.constant_initializer(0))
y_1 = tf.nn.bias_add(tf.matmul(x,w),b)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = y_1))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(y,1),tf.arg_max(y_1,1)),tf.float32))
train_step = tf.train.AdamOptimizer().minimize(loss)
my = tf.arg_max(y_1,1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(501):
        sess.run(train_step,feed_dict = {x:feat_vec,y:label_vec})
        if i % 50 == 0:
            accuracy_print = sess.run(accuracy,feed_dict = {x:feat_vec,y:label_vec})
            print('acc:',accuracy_print)

acc: 0.0106422
acc: 0.782752
acc: 0.984954
acc: 0.999266
acc: 1.0
acc: 1.0
acc: 1.0
acc: 1.0
acc: 1.0
acc: 1.0
acc: 1.0


In [ ]:
with tf.Session() as sess:
    pre = sess.run(my, feed_dict = {x:feat_vec_test})

In [66]:
pre

array([30, 92, 62, 61, 17, 28, 56, 47, 67, 42, 42, 42, 10, 93, 17, 93, 51,
       73, 99, 61, 19, 76, 52, 48, 99,  4,  9, 13, 23, 86,  8, 14, 99, 50,
        9,  3, 51, 57, 95, 58, 37, 71,  8, 42, 76, 77,  1, 89, 89,  8, 40,
        3, 67, 33, 30, 62, 47, 15, 23, 10, 10, 50, 93, 19, 89, 77, 78, 71,
        3, 99, 75, 74, 68, 48, 61, 40, 69,  3, 85, 25,  3, 74, 66, 44, 12,
       71, 85, 25, 85, 42, 62, 71, 26, 99, 66, 78, 84, 90, 86, 22, 81, 33,
       42, 96, 66, 47, 17, 44, 75, 59, 87, 17, 60, 99, 23, 63, 53, 78, 70,
       42, 88,  3, 67, 89, 14, 62, 76, 67, 61, 93, 42, 67, 89, 48, 21,  3,
       61, 15, 15, 97,  8, 40, 93, 89, 88, 40, 21, 61, 52, 58, 61, 58, 58,
        8, 15, 37, 17,  0,  3, 63, 51, 50, 55, 10, 73, 77, 96, 59, 33, 30,
       42, 42, 92, 44, 58, 14, 77, 67, 88, 83, 28, 50, 42, 78,  8,  6, 19,
       69, 30, 25, 26, 96, 89, 67, 12, 90, 67, 33, 42, 13, 70,  8, 42, 78,
       67, 67, 26,  2, 33, 90, 35, 33, 88, 83, 35,  3, 21, 73, 13, 23, 33,
       10, 15, 59, 42, 30

# save answer

In [22]:
ans = []
for i in range(1000):
    ans.append([des_list_test[i][0],pre[i] + 1])
ans

[['0dd7912397dda144a3db0a9eb9b7d0a20df486cf.jpg', 59],
 ['18d8bc3eb13533fa51d47487a2d3fd1f41345b3d.jpg', 66],
 ['00e93901213fb80ec3aeeeca3ad12f2eb93894b4.jpg', 1],
 ['09fa513d269759ee296d4151b9fb43166d22df6e.jpg', 81],
 ['1b4c510fd9f9d72a137df49edf2a2834349bbb1c.jpg', 1],
 ['1e30e924b899a901d6ad172417950a7b0208f50c.jpg', 65],
 ['0b46f21fbe096b63c637676b06338744ebf8acda.jpg', 98],
 ['14ce36d3d539b600070d292de250352ac75cb76c.jpg', 73],
 ['08f790529822720ed5f58c7070cb0a46f21fab05.jpg', 99],
 ['0b46f21fbe096b63e0864f4e00338744ebf8ac14.jpg', 65],
 ['0dd7912397dda1444d90db17b8b7d0a20df486fe.jpg', 35],
 ['0823dd54564e9258fd77debe9782d158ccbf4e1a.jpg', 28],
 ['0b46f21fbe096b639f03b9ee07338744ebf8ac72.jpg', 77],
 ['0b46f21fbe096b63070a21cb07338744ebf8ac1e.jpg', 10],
 ['0e2442a7d933c895679e84e9da1373f0830200bb.jpg', 36],
 ['0824ab18972bd407d35a166170899e510eb30989.jpg', 12],
 ['1b4c510fd9f9d72a0be0fc1bdf2a2834349bbb34.jpg', 9],
 ['09fa513d269759eef4830a58b9fb43166c22df56.jpg', 75],
 ['0b55b319eb

In [25]:
predict = pd.DataFrame(data = ans)
predict.to_csv('predict.csv',header = None,index = False)